In [30]:
%run dataset_preprocessing.ipynb

import argparse
import os
import numpy as np
import math
import sys

import torchvision.transforms as transforms
from torchvision.utils import save_image

from torch.utils.data import DataLoader
from torchvision import datasets
from torch.autograd import Variable

import torch.nn as nn
import torch.nn.functional as F   # NOTE: I don't think this is used
import torch.autograd as autograd
import torch

print(torch.cuda.is_available())

False


In [27]:
class opt():   # Class used for optimizers in the future. Defines all variables and stuff needed.
    save_weights = False
    n_epochs = 20000   # an epoch is the number of times it works through the entire training set
    batch_size = 400   # the training set is broken up into batches, 
                        # and the average loss is used from a given batch for back propagation
    lr = 0.001   # learning rate (how much to change based on error)
    b1 = 0.9   # Used for Adam. Exponential decay rate for the first moment
    b2 = 0.999   # Used for Adam. Exponential decay rate for the second moment estimates (gradient squared)
    input_size = 80
    data_file = "/data"
    config_dir = "."
    save_location = config_dir + data_file
    root_path = "/depot-new/cms/top/mcnama20/TopSpinCorr-Run2-Entanglement/CMSSW_10_2_22/src/TopAnalysis/Configuration/analysis/diLeptonic/three_files"
    file = root_path + "/ee_modified_root_1.root"

    #n_cpu = 2    not used rn


# os.makedirs(config_dir, exit_ok=True)
    
cuda = True if torch.cuda.is_available() else False


class Classifier(nn.Module):
    """
    classifier layers
    """
    def __init__(self):
        super(Classifier, self).__init__()   # Just uses the module constructor with name Discriminator 

        self.model = nn.Sequential(
            nn.Linear(opt.input_size, 3),   # first layer
            nn.LeakyReLU(0.2, inplace=True),   # apply leaky relu to layer
        )

    def forward(self, input):
        """
        applies model to input and attempts to classify
        """
        output = self.model(input)   # Classifies the input (at location) as gg (0) qqbar (1) or other (2)
        return output


# ******* OUT OF CLASSES NOW ************

# Initialize generator and discriminator
classifier = Classifier()

if cuda:
    classifier.cuda()

    

# # Configure data loader - CHANGE
os.makedirs("./data/", exist_ok=True)
dataloader = torch.utils.data.DataLoader(
    ProductionModeDataset(opt.file),
    batch_size=opt.batch_size,
    shuffle=True,
)
print('done')

['ttBar_treeVariables_step8;4;1']
['lb_delta_eta', 'lbbar_delta_eta', 'lnu_delta_eta', 'lnubar_delta_eta', 'lbarb_delta_eta', 'lbarbbar_delta_eta', 'lbarnu_delta_eta', 'lbarnubar_delta_eta', 'bnu_delta_eta', 'bnubar_delta_eta', 'bbarnu_delta_eta', 'bbarnubar_delta_eta', 'lb_delta_phi', 'lbbar_delta_phi', 'lnu_delta_phi', 'lnubar_delta_phi', 'lbarb_delta_phi', 'lbarbbar_delta_phi', 'lbarnu_delta_phi', 'lbarnubar_delta_phi', 'bnu_delta_phi', 'bnubar_delta_phi', 'bbarnu_delta_phi', 'bbarnubar_delta_phi', 'wplusb_delta_eta', 'wplusbbar_delta_eta', 'wminusb_delta_eta', 'wminusbbar_delta_eta', 'wplusb_delta_phi', 'wplusbbar_delta_phi', 'wminusb_delta_phi', 'wminusbbar_delta_phi', 'top_eta', 'top_boosted_eta', 'tbar_eta', 'tbar_boosted_eta', 'ttbar_delta_eta', 'ttbar_eta', 'llbar_delta_eta', 'bbbar_delta_eta', 'nunubar_delta_eta', 'top_phi', 'tbar_phi', 'ttbar_phi', 'ttbar_delta_phi', 'llbar_phi', 'llbar_delta_phi', 'bbbar_phi', 'bbbar_delta_phi', 'nunubar_phi', 'nunubar_delta_phi', 'l_eta', 

In [33]:
optimizer = torch.optim.Adam(classifier.parameters(), lr=opt.lr, betas=(opt.b1, opt.b2)) 
# criterion = nn.MSELoss()   # mean squared error loss
criterion = nn.CrossEntropyLoss()

loss = 0
batches_done = 0   # Counter for batches
for epoch in range(opt.n_epochs):   # Loop through all epochs
    for i, x in enumerate(dataloader): # x is a batch
                                                # there are i batches in the epoch
        
        weight = x[:,81]
        target = x[:,80]
        x = np.transpose(x)
        x = np.delete(x, [80, 81, 82], 0)
        x = np.transpose(x)
        print("mode " + str(target.shape))
        print("x " + str(x.shape))
        
        # Configure input
#         input = Variable(x.type(Tensor))   # Variable is a wrapper for the Tensor x was just made into
        batch = x
        # ---------------------
        #  Train Classifier
        # ---------------------

        optimizer.zero_grad()   # Make gradients zero so they don't accumulate
        
        # apply nn to input
        output = classifier(batch)
        
        # Calculate loss 
        loss = criterion(output, target) # -torch.mean(output) + torch.mean(y) # may need to change datatype of y

        loss.backward()   # Do back propagation 
        optimizer.step()   # Update parameters based on gradients for individuals

        batches_done += 1
        
    if epoch % 10 == 0:
#         np.save('{file_name}/{num_batches}.npy'.format(file_name=opt.data_file, \
#             num_batches=batches_done), loss)
        print(loss)

if opt.save_weights:
    torch.save(classifier.model, opt.config_dir + "/singleLayerModel.pt")




mode torch.Size([400])
x torch.Size([400, 80])


RuntimeError: expected scalar type Float but found Double

In [6]:
############## trying out other losses

loss = nn.CrossEntropyLoss()
input = torch.randn(3, 5, requires_grad=True)
input

tensor([[-0.1354, -0.1125, -0.7349, -0.6369,  0.0472],
        [-0.1748,  1.0386,  0.2885, -0.5644,  0.1862],
        [-1.0530,  1.4145,  1.2992,  1.2871,  1.0882]], requires_grad=True)

In [12]:
target = torch.empty(3, dtype=torch.long).random_(5)
target

tensor([2, 4, 3])

In [8]:
output = loss(input, target)
output

tensor(1.4894, grad_fn=<NllLossBackward>)

In [9]:
output.backward()
output

tensor(1.4894, grad_fn=<NllLossBackward>)